In [2]:
import cv2
import numpy as np
from ..utils.metric_tablebank_my_implementation import metric_table_bank_union
from ..utils.get_models_and_set import get_model, get_1k_test_set, get_test_data_latex, get_test_data_word, get_test_data_publaynet

We save all the image with less than 5% F1 score to explore

In [5]:
take_the_table_bank_model = False
size_model = ""
train_type_name = "word"
test_type_name = "publaynet"

sum_numerator = 0
sum_denominator_precision = 0
sum_denominator_recall = 0

predictor = get_model(train_type_name, size_model, tablebank_model = take_the_table_bank_model)

test_data = []
# Take the WORD data
if "word" in test_type_name:
    word = get_test_data_word()
    test_data.extend(word)
# Take the LATEX data
if "latex" in test_type_name:
    latex = get_test_data_latex()
    test_data.extend(latex)
# Take the PUBLAYNET data
if "publaynet" in test_type_name:
    publaynet = get_test_data_publaynet()
    test_data.extend(publaynet)

print(len(test_data))
for i in range(len(test_data)):

    image_dict = test_data[i]
    try: 
        image_name = "/data/rali5/Tmp/yockelle/TableBank/" + image_dict["file_name"]  
        im = cv2.imread(image_name)
        does_shape_work = im.shape
    except:
        image_name = image_dict["file_name"]
        im = cv2.imread(image_name)

    outputs = predictor(im)

    predictions_detectron = outputs["instances"].pred_boxes.tensor.cpu().numpy()
    scores_detectron = outputs["instances"].scores.cpu().numpy()
    predictions_with_threshold = []
    for j, p in enumerate(predictions_detectron):
        if scores_detectron[j]>0.9:
            predictions_with_threshold.append(p)
    predictions_with_threshold = np.array(predictions_with_threshold)

    # Get the ground truth
    bbox_ground_truth = []
    truth = image_dict["annotations"]
    for t in truth:
        x_min = t["bbox"][0]
        y_min = t["bbox"][1]
        w = t["bbox"][2]
        h = t["bbox"][3]
        x_max = x_min + w
        y_max = y_min + h
        cv2.rectangle(im, (int(x_min), int(y_max)), (int(x_max), int(y_min)), (0,255,0), 3)
        bbox_ground_truth.append([x_min, y_min, x_max, y_max])

    # get metrics Detectron
    metric_result = metric_table_bank_union(bbox_ground_truth, predictions_with_threshold)

    if metric_result != None:
        numerator, denominator_precision, denominator_recall, old_way_area_union = metric_result

        sum_numerator += numerator
        sum_denominator_precision += denominator_precision
        sum_denominator_recall += denominator_recall

        if denominator_precision>0:
            precision = numerator/denominator_precision
        else:
            precision = 0
        if denominator_recall > 0:
            recall = numerator/denominator_recall
        else:
            recall = 0
        if recall==0 and precision==0:
            f1=0
        else:
            f1 = (2 * precision * recall) / (precision + recall)
        
        if f1 <= 0.05:
            only_name = image_name.split("/")[-1]
            cv2.imwrite("images_problem_"+ test_type_name +"/" + only_name + ".png", im)

#print("Precision ", precision)
#print("Recall ", recall)
#print("F1 ", f1)

./output/word/ model_final.pth
5719
